## Постоение объяснений работы модели при помощи PyG и Captum

Решаем задачу классификации графов на примере датасета Mutagenicity

In [3]:
# несколько удобных функций для описания датасетов
def describe_dataset(dataset):
    print(f'Dataset: {dataset}:')
    print('======================')
    print(f'Number of graphs: {len(dataset)}')
    print(f'Number of features: {dataset.num_features}')
    print(f'Number of classes: {dataset.num_classes}')

def describe_graph(g):
    print(g)
    print('==============================================================')

    # Gather some statistics about the graph.
    print(f'Number of nodes: {g.num_nodes}')
    print(f'Number of edges: {g.num_edges}')
    print(f'Average node degree: {g.num_edges / g.num_nodes:.2f}')
    if hasattr(g, 'train_mask'):
        print(f'Number of training nodes: {g.train_mask.sum()}')
        print(f'Training node label rate: {int(g.train_mask.sum()) / g.num_nodes:.2f}')
    print(f'Has isolated nodes: {g.has_isolated_nodes()}')
    print(f'Has self-loops: {g.has_self_loops()}')
    print(f'Is undirected: {g.is_undirected()}')

In [1]:
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import TUDataset

dataset = TUDataset('./tmp/mutag', name='Mutagenicity').shuffle()
test_dataset = dataset[:len(dataset) // 10]
train_dataset = dataset[len(dataset) // 10:]
test_loader = DataLoader(test_dataset, batch_size=128)
train_loader = DataLoader(train_dataset, batch_size=128)

E:\anaconda3\envs\gcn\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Extracting tmp\mutag\Mutagenicity\Mutagenicity.zip
Processing...
Done!


In [4]:
describe_dataset(dataset)

Dataset: Mutagenicity(4337):
Number of graphs: 4337
Number of features: 14
Number of classes: 2


In [5]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch_geometric.nn as gnn
import torch.nn.functional as F

class GCN(nn.Module):
    def __init__(
        self, 
        n_input: int, 
        n_hidden_layers: int, 
        n_hidden: int, 
        n_out: int,
        n_classes: int,
        dropout_p: float = 0.2,
        activation: callable = F.relu
    ) -> None:
        super().__init__()
        self.dropout_p = dropout_p
        self.activation = activation
        # подход к построению глубоких GNN взят отсюда: 
        # https://github.com/dmlc/dgl/blob/master/examples/pytorch/graphsage/advanced/model.py
        self.layers = nn.ModuleList()
        # в качестве слоев используется gnn.GraphConv
        # т.к. он может работать с весами на ребрах
        if n_hidden_layers > 1:
            self.layers.append(gnn.GraphConv(n_input, n_hidden))
            for _ in range(1, n_hidden_layers-1):
                self.layers.append(gnn.GraphConv(n_hidden, n_hidden))
            self.layers.append(gnn.GraphConv(n_hidden, n_out))
        else:
            self.layers.append(gnn.GraphConv(n_input, n_out))
        self.classifier = nn.Linear(n_out, n_classes)

    def forward(self, x, edge_index, batch, edge_weight=None):
        # 1. Получение эмбеддингов узлов
        h = x
        for layer in self.layers:
            h = layer(h, edge_index, edge_weight)
            h = self.activation(h)
            h = F.dropout(h, p=self.dropout_p, training=self.training)
        
        # 2. Агрегация
        h = gnn.global_add_pool(h, batch)
        h = F.dropout(h, p=self.dropout_p, training=self.training)
        # 3. Полносвязный слой для классификации графа
        h = self.classifier(h)
        return h

In [8]:
model = GCN(
    dataset.num_features, 
    n_hidden_layers=5, 
    n_hidden=32,
    n_out=32, 
    n_classes=dataset.num_classes,
    activation=torch.relu,
    dropout_p=0.5
)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01, weight_decay=5e-4)

for epoch in range(31):
    epoch_losses = []
    epoch_acc_train = 0
    epoch_acc_test = 0
    # train
    for step, data in enumerate(train_loader):  # Итерируемся по пакетам в обучающей выборке.
        logits = model(data.x, data.edge_index, data.batch)  
        loss = criterion(logits, data.y)
        loss.backward()  
        optimizer.step()
        optimizer.zero_grad()

        epoch_losses.append(loss.item())
        epoch_acc_train += (logits.argmax(dim=1) == data.y).sum().item()
    epoch_acc_train /= len(train_loader.dataset)
    # eval test
    for data in test_loader:
        logits = model(data.x, data.edge_index, data.batch)  
        epoch_acc_test += (logits.argmax(dim=1) == data.y).sum().item()
    epoch_acc_test /= len(test_loader.dataset)


    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d} Avg Loss: {np.mean(epoch_losses):.4f} '
              f'Train Acc: {epoch_acc_train:.4f} Test Acc: {epoch_acc_test:.4f}')

GCN(
  (layers): ModuleList(
    (0): GraphConv(14, 32)
    (1): GraphConv(32, 32)
    (2): GraphConv(32, 32)
    (3): GraphConv(32, 32)
    (4): GraphConv(32, 32)
  )
  (classifier): Linear(in_features=32, out_features=2, bias=True)
)
Epoch: 000 Avg Loss: 1.8125 Train Acc: 0.5051 Test Acc: 0.5104
Epoch: 010 Avg Loss: 0.6121 Train Acc: 0.6657 Test Acc: 0.6721
Epoch: 020 Avg Loss: 0.5876 Train Acc: 0.7003 Test Acc: 0.7275


Существует несколько подходов для объяснений предсказаний. 

### Saliency method 
Нужно рассчитать модуль градиента 
$$
Attribution_{e_i} = |\frac{\partial F(x)}{\partial w_{e_i}}|
$$
где $x$ - это входные данные, а $F(x)$ - результат работы модели на входе $x$

### Integrated Gradients method
Нужно рассчитать интеграл
$$
Attribution_{e_i} = \int_{\alpha =0}^1 \frac{\partial F(x_{\alpha)}}{\partial w_{e_i}} d\alpha
$$
где $x_{\alpha}$ - исходный граф, где веса всех ребер заменены на $\alpha$.

Для расчета этих значений можно воспользоваться пакетом `captum`.

In [11]:
from captum.attr import Saliency, IntegratedGradients

def model_forward(edge_mask, data):
    # модель работает с пакетами, но в этой функции
    # мы работаем с одним графом, поэтому фейкаем тензор
    # batch
    batch = torch.zeros(data.x.shape[0], dtype=int)
    # edge_weight - это веса на ребрах
    out = model(data.x, data.edge_index, batch, edge_mask)
    return out

def explain(data, target=0):
    input_mask = torch.ones(data.edge_index.shape[1]).requires_grad_(True)
    ig = IntegratedGradients(model_forward)
    mask = ig.attribute(input_mask, target=target,
                        additional_forward_args=(data,),
                        internal_batch_size=data.edge_index.shape[1])
    edge_mask = np.abs(mask.cpu().detach().numpy())
    if edge_mask.max() > 0:  # avoid division by zero
        edge_mask = edge_mask / edge_mask.max()
    return edge_mask

In [26]:
from random import choice
import pandas as pd
data = choice([t for t in test_dataset if not t.y.item()])
edge_mask = explain(data, target=0)

df = pd.DataFrame(zip(*data.edge_index.numpy(), edge_mask), columns=['u', 'v', 'importance'])
df.sort_values(by='importance', ascending=False)

,u,v,importance
11,3,8,1.000000
9,3,0,0.921472
10,3,7,0.897662
19,7,3,0.809852
20,8,3,0.768895
16,6,2,0.279283
1,0,2,0.187710
0,0,1,0.168890
6,2,0,0.166630
21,9,4,0.157605
